# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [8]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [9]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [11]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [12]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
          'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
g_s_svd.fit(data)

In [13]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8703402526657775, 'mae': 0.6687856625868747}
{'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [16]:
# cross validating with KNNBasic
sim_pearson = {'name': 'pearson', 'user_based':True}
knn_basic = KNNBasic(sim_options=sim_pearson)
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [18]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97333219, 0.96689451, 0.98171821, 0.97581614, 0.96391462]))
('test_mae', array([0.7527933 , 0.74757757, 0.75682587, 0.75073222, 0.74577954]))
('fit_time', (0.4214468002319336, 0.5221600532531738, 0.4324648380279541, 0.7427840232849121, 0.40334320068359375))
('test_time', (1.2431681156158447, 1.351794958114624, 1.4847369194030762, 1.2406196594238281, 1.0766689777374268))
-----------------------
0.9723351348065978


In [19]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [20]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87475615, 0.87850676, 0.87505949, 0.87714762, 0.87863916]))
('test_mae', array([0.67018508, 0.67021849, 0.66764677, 0.67036947, 0.67224983]))
('fit_time', (0.5665562152862549, 0.6060972213745117, 0.5582828521728516, 0.5625011920928955, 0.5684669017791748))
('test_time', (1.9576959609985352, 1.6843299865722656, 1.7454981803894043, 1.6278679370880127, 1.7968177795410156))


0.8768218346677342

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [21]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [22]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [23]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [25]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0570465346081006, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [26]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list  

In [27]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Comedy')


      movieId                 title  genres
1370     1875  Clockwatchers (1997)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                  title                  genres
3075     4128  Lost Boys, The (1987)  Comedy|Horror|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                title      genres
4866     7292  Best Defense (1984)  Comedy|War
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                                              title  genres
6310    48385  Borat: Cultural Learnings of America for Make ...  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [28]:
user_rating

[{'userId': 1000, 'movieId': 1875, 'rating': '5'},
 {'userId': 1000, 'movieId': 4128, 'rating': '4'},
 {'userId': 1000, 'movieId': 7292, 'rating': '4'},
 {'userId': 1000, 'movieId': 48385, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [29]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [30]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors = 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [32]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []

for m_id in new_df['movieId'].unique():
    list_of_movies.append((m_id, svd_.predict(1000,m_id)[3]))

In [33]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [38]:
ranked_movies

[(750, 4.523966598056141),
 (858, 4.4797184767936225),
 (1204, 4.479425264202948),
 (904, 4.478191976448613),
 (318, 4.474128028047394),
 (1136, 4.4381905958714505),
 (1104, 4.435804575104071),
 (922, 4.433373122745514),
 (720, 4.414649211291949),
 (56782, 4.412668951680975),
 (1248, 4.4050232458583505),
 (2019, 4.400558830375842),
 (2324, 4.399993003515867),
 (898, 4.397138356068665),
 (7153, 4.39557396114467),
 (1267, 4.392682479343103),
 (2959, 4.391367573928313),
 (1196, 4.3907955505346346),
 (4973, 4.389620137789419),
 (1089, 4.3862047120904775),
 (1221, 4.384192225159463),
 (48516, 4.383245968055279),
 (908, 4.383143043502418),
 (1237, 4.3827411292843275),
 (1252, 4.3802462202622365),
 (930, 4.378960887061773),
 (27773, 4.377028105549103),
 (1213, 4.376871696842895),
 (3451, 4.3738310674911265),
 (4226, 4.36968280482012),
 (246, 4.366825016255949),
 (7361, 4.3644800267849915),
 (1276, 4.363157226341959),
 (1288, 4.362130706665531),
 (912, 4.361538823556896),
 (1208, 4.35613120443

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [39]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ':', rec[0], title, '\n')
            n-=1
            if n == 0:
                break
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 : 750 602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 : 858 659    Godfather, The (1972)
Name: title, dtype: object 

Recommendation #  3 : 1204 906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  4 : 904 686    Rear Window (1954)
Name: title, dtype: object 

Recommendation #  5 : 318 277    Shawshank Redemption, The (1994)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.